In [1]:
!pipenv install pyspark

Installing pyspark...

Installing dependencies from Pipfile.lock (3a1865)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.



[    ] Installing...
[=   ] Installing pyspark...
[==  ] Installing pyspark...
[=== ] Installing pyspark...
[ ===] Installing pyspark...
[  ==] Installing pyspark...
[   =] Installing pyspark...
[    ] Installing pyspark...
[   =] Installing pyspark...
[  ==] Installing pyspark...
[ ===] Installing pyspark...
[====] Installing pyspark...
[=== ] Installing pyspark...
[==  ] Installing pyspark...
[=   ] Installing pyspark...
[    ] Installing pyspark...
[=   ] Installing pyspark...
[==  ] Installing pyspark...
[=== ] Installing pyspark...
[ ===] Installing pyspark...
[  ==] Installing pyspark...
[   =] Installing pyspark...
[    ] Installing pyspark...
[   =] Installing pyspark...
[  ==] Installing pyspark...
[ ===] Installing pyspark...
[====] Installing pyspark...
[=== ] Installing pyspark...
[==  ] Installing pyspark...
[=   ] Installing pyspark...
[    ] Installing pyspark...
[=   ] Installing pyspark...
[==  ] Installing pyspark...
Adding pyspark to 

In [2]:
import pyspark
import pandas as pd
import pyspark.sql
import matplotlib.pyplot as plt
import seaborn as sns
import time
import findspark
from pyspark.ml.feature import Tokenizer, RegexTokenizer,IDF
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
%matplotlib inline
from pyspark.ml.feature import CountVectorizer
from pyspark import SparkContext

In [3]:
findspark.init(r"C:\Users\rejee\spark-3.3.1-bin-hadoop3")

In [4]:
sc = SparkContext(master="local[1]")

In [5]:
sc

<SparkContext master=local[1] appName=pyspark-shell>

In [6]:
from pyspark.sql import SparkSession

In [69]:
spark=SparkSession.builder.appName("Amazone").getOrCreate()

In [8]:
spark


In [9]:
df = spark.read.load(".\kindle_reviews.csv", format="csv", sep=",", header="true", inferSchema=True)

In [10]:
df.show()

+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|      asin|helpful|overall|          reviewText|          reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|B000F83SZQ| [0, 0]|      5|I enjoy vintage b...|          05 5, 2014|      A1F6404F1VG29J|          Avidreader|  Nice vintage story|          1399248000|
|  1|B000F83SZQ| [2, 2]|      4|This book is a re...|          01 6, 2014|       AN0N05A9LIJEQ|            critters|        Different...|          1388966400|
|  2|B000F83SZQ| [2, 2]|      4|This was a fairly...|          04 4, 2014|       A795DMNCJILA6|                 dot|               Oldie|          1396569600|
|  3|B000F83SZQ| [1, 1]|      5|I'd never read

In [11]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- helpful: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: string (nullable = true)



In [12]:
df.show()

+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|      asin|helpful|overall|          reviewText|          reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|B000F83SZQ| [0, 0]|      5|I enjoy vintage b...|          05 5, 2014|      A1F6404F1VG29J|          Avidreader|  Nice vintage story|          1399248000|
|  1|B000F83SZQ| [2, 2]|      4|This book is a re...|          01 6, 2014|       AN0N05A9LIJEQ|            critters|        Different...|          1388966400|
|  2|B000F83SZQ| [2, 2]|      4|This was a fairly...|          04 4, 2014|       A795DMNCJILA6|                 dot|               Oldie|          1396569600|
|  3|B000F83SZQ| [1, 1]|      5|I'd never read

In [13]:
df=df.drop('reviewTime','reviewerName','reviewerID','helpful','unixReviewTime')

In [14]:
df.show()

+---+----------+-------+--------------------+--------------------+
|_c0|      asin|overall|          reviewText|             summary|
+---+----------+-------+--------------------+--------------------+
|  0|B000F83SZQ|      5|I enjoy vintage b...|  Nice vintage story|
|  1|B000F83SZQ|      4|This book is a re...|        Different...|
|  2|B000F83SZQ|      4|This was a fairly...|               Oldie|
|  3|B000F83SZQ|      5|I'd never read an...|  I really liked it.|
|  4|B000F83SZQ|      4|If you like perio...|      Period Mystery|
|  5|B000F83SZQ|      4|A beautiful in-de...|              Review|
|  6|B000F83SZQ|      4|I enjoyed this on...|Nice old fashione...|
|  7|B000F83SZQ|      4|Never heard of Am...|Enjoyable reading...|
|  8|B000FA64PA|      5|Darth Maul workin...|          Darth Maul|
|  9|B000FA64PA|      4|This is a short s...|Not bad, not exce...|
| 10|B000FA64PA|      5|I think I have th...|      Audio and book|
| 11|B000FA64PA|      4|Title has nothing...|Darth Maul...the 

In [15]:
df1=df.groupBy('overall').count().show()


+-------+------+
|overall| count|
+-------+------+
|   null|   201|
|      1| 23018|
|      3| 96194|
|      5|575264|
|      4|254013|
|      2| 34130|
+-------+------+



In [16]:
df=df.select('summary','overall')

In [17]:
df.show()

+--------------------+-------+
|             summary|overall|
+--------------------+-------+
|  Nice vintage story|      5|
|        Different...|      4|
|               Oldie|      4|
|  I really liked it.|      5|
|      Period Mystery|      4|
|              Review|      4|
|Nice old fashione...|      4|
|Enjoyable reading...|      4|
|          Darth Maul|      5|
|Not bad, not exce...|      4|
|      Audio and book|      5|
|Darth Maul...the ...|      4|
|Not bad; it is we...|      3|
|Han and Leia reun...|      3|
|  Possibly Important|      5|
|        Another read|      5|
|            Recovery|      5|
|Star Wars: The Ne...|      3|
|       my collection|      5|
| where it's bille...|      4|
+--------------------+-------+
only showing top 20 rows



In [18]:
df.groupBy('overall').count().show()

+-------+------+
|overall| count|
+-------+------+
|   null|   201|
|      1| 23018|
|      3| 96194|
|      5|575264|
|      4|254013|
|      2| 34130|
+-------+------+



In [67]:
df=df.groupBy('overall').count().show()

AttributeError: 'NoneType' object has no attribute 'groupBy'

In [19]:
df.toPandas()['overall'].value_counts()

5.0    575264
4.0    254013
3.0     96194
2.0     34130
1.0     23018
Name: overall, dtype: int64

In [20]:
df.toPandas()['overall'].isnull().sum()

201

In [21]:
df.toPandas()['summary'].isnull().sum()

435

In [22]:
df = df.na.drop(subset=['summary', 'overall'])

In [23]:
df.toPandas()['overall'].isnull().sum()

0

In [24]:
df.toPandas()['summary'].isnull().sum()

0

In [25]:
df.show(5)

+------------------+-------+
|           summary|overall|
+------------------+-------+
|Nice vintage story|      5|
|      Different...|      4|
|             Oldie|      4|
|I really liked it.|      5|
|    Period Mystery|      4|
+------------------+-------+
only showing top 5 rows



In [26]:
import pyspark.ml.feature

In [27]:
from pyspark.ml.feature import StringIndexer

In [28]:
tokenizer=Tokenizer(inputCol='summary',outputCol='review_token')
stop_words=StopWordsRemover(inputCol='review_token',outputCol='review_stop')
countvector=CountVectorizer(inputCol='review_stop',outputCol='review_count')
Idf=IDF(inputCol='review_count',outputCol='Idf_features')

In [29]:
labelEncoder = StringIndexer(inputCol='overall', outputCol='label').fit(df)

In [30]:
labelEncoder.transform(df).show(5)

+------------------+-------+-----+
|           summary|overall|label|
+------------------+-------+-----+
|Nice vintage story|      5|  0.0|
|      Different...|      4|  1.0|
|             Oldie|      4|  1.0|
|I really liked it.|      5|  0.0|
|    Period Mystery|      4|  1.0|
+------------------+-------+-----+
only showing top 5 rows



In [31]:
labelEncoder.labels

['5', '4', '3', '2', '1']

In [32]:
df.show()

+--------------------+-------+
|             summary|overall|
+--------------------+-------+
|  Nice vintage story|      5|
|        Different...|      4|
|               Oldie|      4|
|  I really liked it.|      5|
|      Period Mystery|      4|
|              Review|      4|
|Nice old fashione...|      4|
|Enjoyable reading...|      4|
|          Darth Maul|      5|
|Not bad, not exce...|      4|
|      Audio and book|      5|
|Darth Maul...the ...|      4|
|Not bad; it is we...|      3|
|Han and Leia reun...|      3|
|  Possibly Important|      5|
|        Another read|      5|
|            Recovery|      5|
|Star Wars: The Ne...|      3|
|       my collection|      5|
| where it's bille...|      4|
+--------------------+-------+
only showing top 20 rows



In [33]:
df = labelEncoder.transform(df)

In [34]:
df.show(5)

+------------------+-------+-----+
|           summary|overall|label|
+------------------+-------+-----+
|Nice vintage story|      5|  0.0|
|      Different...|      4|  1.0|
|             Oldie|      4|  1.0|
|I really liked it.|      5|  0.0|
|    Period Mystery|      4|  1.0|
+------------------+-------+-----+
only showing top 5 rows



In [35]:
(trainDF,testDF)=df.randomSplit((0.7,0.3),seed=42)


In [36]:
trainDF.show(5)

+--------------------+-------+-----+
|             summary|overall|label|
+--------------------+-------+-----+
|  Cats and romanc...|      5|  0.0|
|       Charles Vance|      2|  3.0|
|    Have a nice day"|      5|  0.0|
|  I can see a tre...|      2|  3.0|
|  I have trouble ...|      5|  0.0|
+--------------------+-------+-----+
only showing top 5 rows



In [37]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
#df1=LabelEncoder.transform(df)

In [39]:
lr=LogisticRegression(featuresCol='Idf_features',labelCol='label')

In [40]:
pipeline = Pipeline(stages=[tokenizer,stop_words,countvector,Idf,lr])

In [41]:
pipeline

Pipeline_f6cd4ec39cb7

In [42]:
pipeline.stages

Param(parent='Pipeline_f6cd4ec39cb7', name='stages', doc='a list of pipeline stages')

In [43]:
# Building MOdel
lr_model = pipeline.fit(trainDF)

In [44]:
lr_model

PipelineModel_a0f4533ddde9

In [45]:
# Predictions on our Test Dataset
predictions = lr_model.transform(testDF)

In [46]:
predictions.show()

+--------------------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|             summary|overall|label|        review_token|         review_stop|        review_count|        Idf_features|       rawPrediction|         probability|prediction|
+--------------------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|              Hatrid|      4|  1.0|        [, , hatrid]|        [, , hatrid]|  (147807,[3],[2.0])|(147807,[3],[5.78...|[1.25459471351851...|[0.48018102212748...|       0.0|
|  I look forward ...|      5|  0.0|[, , i, look, for...|[, , look, forwar...|(147807,[3,89,204...|(147807,[3,89,204...|[6.53120992248003...|[0.03847637205656...|       2.0|
|               Keta |      5|  0.0|          [, , keta]|          [, , keta]|(147807,[3,24932]...|(147807,[3,24932]...|[7.3833363

In [47]:
# Select Columns
predictions.columns

['summary',
 'overall',
 'label',
 'review_token',
 'review_stop',
 'review_count',
 'Idf_features',
 'rawPrediction',
 'probability',
 'prediction']

In [48]:
predictions.select('rawPrediction','probability','summary','label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|       rawPrediction|         probability|             summary|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|[1.25459471351851...|[0.48018102212748...|              Hatrid|  1.0|       0.0|
|[6.53120992248003...|[0.03847637205656...|  I look forward ...|  0.0|       2.0|
|[7.38333631850203...|[0.99954584886883...|               Keta |  0.0|       0.0|
|[11.0307083234675...|[0.49435966749339...|  a Semel will ch...|  0.0|       1.0|
|[-0.3025810869600...|[5.91574515444475...|  and it was too ...|  2.0|       2.0|
|[5.14255271794278...|[6.07968985632261...|  and kills some ...|  3.0|       1.0|
|[19.2069723660228...|[0.99999664346831...|  and who owned a...|  4.0|       0.0|
|[11.2788161050692...|[7.47229195248579...|  his mate works ...|  1.0|       2.0|
|[12.0376571970278...|[7.65935705387111...|  plus he purpose...|  1.0|       1.0|
|[7.988610885560

In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

In [51]:
accuracy = evaluator.evaluate(predictions)

In [52]:
accuracy

0.6128843875094643

In [53]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [54]:
lr_metric = MulticlassMetrics(predictions['label','prediction'].rdd)

C:\Users\rejee\anaconda3\lib\site-packages\pyspark\sql\context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [55]:
print("Accuracy:",lr_metric.accuracy)
print("Precision:",lr_metric.precision(1.0))
print("Recall:",lr_metric.recall(1.0))
print("F1Score:",lr_metric.fMeasure(1.0))

Accuracy: 0.6128843875094643
Precision: 0.30615117472177644
Recall: 0.43436794266410345
F1Score: 0.3591595484463375


In [56]:
y_true = predictions.select('label')
y_true = y_true.toPandas()
y_pred = predictions.select('prediction')
y_pred = y_pred.toPandas()

In [57]:
from sklearn.metrics import confusion_matrix,classification_report

In [58]:
cm = confusion_matrix(y_true,y_pred)

In [59]:
cm

array([[145636,  20873,   3635,   1249,   1050],
       [ 46901,  23273,   4392,    868,    584],
       [ 11239,   7579,   7858,   1456,    669],
       [  4080,   1226,   2182,   1773,   1044],
       [  2616,    628,    772,    973,   1971]], dtype=int64)